In [1]:
from __future__ import annotations

import io
import logging
from typing import Iterable, Optional, Union

import pandas as pd
import requests

In [2]:
logger = logging.getLogger("fred_api")
logger.setLevel(logging.INFO)

# Remove existing handlers
for handler in logger.handlers[:]:
    logger.removeHandler(handler)

# File handler: logs all INFO+ to file
file_handler = logging.FileHandler("fred_logs.txt")
file_handler.setLevel(logging.INFO)
file_formatter = logging.Formatter(
    "%(asctime)s %(levelname)s [%(name)s]: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)
file_handler.setFormatter(file_formatter)
logger.addHandler(file_handler)

# Console handler: logs only WARNING+ to console 
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.WARNING)
console_formatter = logging.Formatter(
    "%(levelname)s: %(message)s"
)
console_handler.setFormatter(console_formatter)
logger.addHandler(console_handler)

# Prevent propagation to root logger
logger.propagate = False

print("Hybrid logging enabled: Warnings/errors in console, all logs in fred_logs.txt")


Hybrid logging enabled: Warnings/errors in console, all logs in fred_logs.txt


In [3]:
FRED_BASE = "https://fred.stlouisfed.org/graph/fredgraph.csv?id="

# Default logger
_default_logger = logging.getLogger("fred_api")
if not _default_logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter(
        "%(asctime)s %(levelname)s [%(name)s]: %(message)s", datefmt="%Y-%m-%d %H:%M:%S"
    )
    handler.setFormatter(formatter)
    _default_logger.addHandler(handler)
_default_logger.setLevel(logging.INFO)


# FRED loader
def get_fred_series(series_id: str) -> pd.DataFrame:
    """
    Fetch a FRED series using the fredgraph.csv endpoint.
    """
    url = f"https://fred.stlouisfed.org/graph/fredgraph.csv?id={series_id}"

    try:
        df = pd.read_csv(url)
    except Exception as exc:
        raise ConnectionError(f"Failed to download FRED series '{series_id}'.") from exc

    # Convert columns
    if "observation_date" not in df or series_id not in df:
        raise ValueError(f"Unexpected CSV format received for series '{series_id}'.")

    df["observation_date"] = pd.to_datetime(df["observation_date"], errors="raise")
    df[series_id] = pd.to_numeric(df[series_id], errors="coerce")

    df = df.rename(columns={"observation_date": "date", series_id: "value"})
    df = df.sort_values("date").reset_index(drop=True)

    return df


# Main API function

def FredAPI(
    series_id: str,
    dates: Optional[Iterable[Union[str, pd.Timestamp]]] = None,
    start_date: Optional[str] = None,
    end_date: Optional[str] = None,
    timeout: int = 15,
    session: Optional[requests.Session] = None,
    logger: Optional[logging.Logger] = None,
) -> pd.DataFrame:

    log = logger or _default_logger

    # Helpers 
    def _validate_input_modes(_dates, _start_date, _end_date):
        if _dates is not None and (_start_date is not None or _end_date is not None):
            raise ValueError("Pass either 'dates' OR ('start_date'/'end_date'), not both.")

    def _validate_types(_series_id, _dates, _start_date, _end_date):
        if not isinstance(_series_id, str) or not _series_id.strip():
            raise TypeError("series_id must be a non-empty string.")
        if _dates is not None and not isinstance(_dates, Iterable):
            raise TypeError("'dates' must be an iterable.")
        if _start_date is not None and not isinstance(_start_date, str):
            raise TypeError("'start_date' must be a string YYYY-MM-DD.")
        if _end_date is not None and not isinstance(_end_date, str):
            raise TypeError("'end_date' must be a string YYYY-MM-DD.")

    def _parse_dates_iterable(_dates):
        try:
            parsed = pd.to_datetime(list(_dates), errors="raise")
        except Exception as exc:
            raise ValueError("One or more items in 'dates' could not be parsed.") from exc
        return pd.DatetimeIndex(parsed.normalize())

    def _parse_single_date(date_str):
        try:
            ts = pd.to_datetime(date_str, format="%Y-%m-%d", errors="raise").normalize()
        except Exception as exc:
            raise ValueError(f"Date '{date_str}' must be YYYY-MM-DD.") from exc
        return ts

    # Validate inputs 
    _validate_input_modes(dates, start_date, end_date)
    _validate_types(series_id, dates, start_date, end_date)

    # download data
    df = get_fred_series(series_id)

    if df.empty:
        raise ValueError(f"Series '{series_id}' returned no rows.")
    if df["value"].isna().all():
        raise ValueError(f"All values for series '{series_id}' are NaN.")

    # Routing
    if dates is None and start_date is None and end_date is None:
        return df

    min_date = df["date"].min()
    max_date = df["date"].max()

    # Handle specific dates 
    if dates is not None:
        requested = _parse_dates_iterable(dates)

        too_early = requested[requested < min_date]
        if len(too_early) > 0:
            raise ValueError(
                f"Requested {len(too_early)} date(s) before series inception ({min_date.date()})."
            )

        out = (
            df.set_index("date")
            .reindex(requested)
            .ffill()
            .reset_index()
            .rename(columns={"index": "date"})
        )

        missing_count = (requested.difference(df["date"])).size
        if missing_count > 0:
            log.warning(
                f"{missing_count} requested date(s) missing in source; forward-filled."
            )

        return out

    # Handle ranges
    start_ts = _parse_single_date(start_date) if start_date else min_date
    end_ts = _parse_single_date(end_date) if end_date else max_date

    if start_ts > end_ts:
        raise ValueError(f"start_date {start_ts.date()} > end_date {end_ts.date()}.")
        log.warning(f"ValueError:(start_date {start_ts.date()} > end_date {end_ts.date()}.")

    if start_ts < min_date:
        raise ValueError(
            f"start_date {start_ts.date()} is before series inception ({min_date.date()})."
        )
        log.warning(f"start_date {start_ts.date()} is before series inception ({min_date.date()}).")

    daily = pd.date_range(start_ts, end_ts, freq="D")

    out = (
        df.set_index("date")
        .reindex(daily)
        .ffill()
        .reset_index()
        .rename(columns={"index": "date"})
    )

    missing_count = (daily.difference(df["date"])).size
    if missing_count > 0:
        log.warning(f"{missing_count} day(s) missing in source; forward-filled.")

    return out


In [4]:
df = FredAPI("DTB3")
print(df.head())

        date  value
0 1954-01-04   1.33
1 1954-01-05   1.28
2 1954-01-06   1.28
3 1954-01-07   1.31
4 1954-01-08   1.31


In [5]:
df = FredAPI("DTB3", start_date="1954-01-01")
df

ValueError: start_date 1954-01-01 is before series inception (1954-01-04).

In [6]:
df = FredAPI("DTB3", end_date="2025-12-01")
df

,date,value
0,1954-01-04,1.33
1,1954-01-05,1.28
2,1954-01-06,1.28
3,1954-01-07,1.31
4,1954-01-08,1.31
...,...,...
26260,2025-11-27,3.75
26261,2025-11-28,3.75
26262,2025-11-29,3.75
26263,2025-11-30,3.75


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26265 entries, 0 to 26264
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    26265 non-null  datetime64[ns]
 1   value   26265 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 410.5 KB


In [ ]:
# 1) Full series (e.g., 3-Month Treasury Bill: DTB3)
df_full = FredAPI("DTB3")
print(df_full.head())

In [8]:
# 2) Specific dates 
df_dates = FredAPI("DGS2", dates=["2024-12-30", "2024-12-31", "2025-01-01"])
print(df_dates)

        date  value
0 2024-12-30   4.24
1 2024-12-31   4.25
2 2025-01-01   4.25


In [9]:
# 3) Start/end range (daily; forward-filled)
df_range = FredAPI("DGS10", start_date="2025-01-01", end_date="2025-02-15")
print(df_range.tail())

         date  value
41 2025-02-11   4.54
42 2025-02-12   4.62
43 2025-02-13   4.52
44 2025-02-14   4.47
45 2025-02-15   4.47


In [10]:
# 4) Error: unknown series
try:
    FredAPI("XXXX")
except ValueError as e:
    print("Caught expected error:", e)

ConnectionError: Failed to download FRED series 'XXXX'.

In [11]:
# 5) Error: requested dates before inception
try:
    FredAPI("UNRATE", ["1900-01-01"])
except ValueError as e:
    print("Caught expected error:", e)

Caught expected error: Requested 1 date(s) before series inception (1948-01-01).
